In [18]:
import numpy as np
import nltk
from nltk.text import TextCollection
from pymongo import MongoClient
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import Preprocessor
from bson import ObjectId
import operator
import sklearn.metrics.pairwise as sklearnPairwise
from scipy.sparse import hstack,vstack
from scipy.sparse import csr_matrix

In [19]:
client = MongoClient("localhost")
db=client.crawlerdb_WORK_TFIDF_3
collection = db.crawlerdb
result_documents_list_REAL = list(collection.find({})) # work-data-collection
print("We got ", len(result_documents_list_REAL), " Documents")

We got  37821  Documents


In [20]:
preprocessor = Preprocessor.Preprocessor()

In [21]:
#result_document_text_list = [a["text"] for a in result_documents_list_REAL] # wir müssen erst result_documents_list["text"] "entpacken" von cursor
result_document_text_list = list()
for i in range(len(result_documents_list_REAL)):
    try:
        result_document_text_list.append(result_documents_list_REAL[i]["text"]) # Fehler: KeyError: muss abgefangen werden
    except KeyError:
        pass
print("We still have ", len(result_document_text_list), " Documents")

We still have  37821  Documents


In [22]:
tfidf = TfidfVectorizer()
tfidf_values = tfidf.fit_transform(result_document_text_list) # output= <class 'scipy.sparse.csr.csr_matrix'>

In [23]:
type(tfidf_values)

scipy.sparse.csr.csr_matrix

In [24]:
# erstellt ein Dict mit {idOfDocument(andNotMore): tf-idf-value}, wobei tf-idf-value eine sparse matrix mit 1xanzahlUniquerWörter ist
result_dict_with_ids = {}
for i in range(len(result_documents_list_REAL)):
    key = str(result_documents_list_REAL[i]["_id"]) # 5c2fb3c988df422822370767
    result_dict_with_ids[key] = tfidf_values[i]

In [25]:
result_dict_with_ids["5c2fb3c988df422822370767"] # with 65 stored elements in Compressed Sparse Row format> means that 65 times is not a 0
shape_of_one_document = result_dict_with_ids["5c2fb3c988df422822370767"].shape

### finished pre TF_IDF - Calculation here

--------------------------------

1

In [29]:
%%time
search_term = "haus"
tokenized_words = preprocessor.tokenizing_without_punc(search_term)
stemmed_search_term = preprocessor.stemming_words(tokenized_words)

tfidf_search_term = tfidf.transform([search_term]) # jetzt brauchen wir ja wieder den "richtigen" searchterm zum Vergleich # transform passt idf nicht an


print(stemmed_search_term)

['haus']
CPU times: user 3.76 ms, sys: 266 µs, total: 4.02 ms
Wall time: 5.36 ms


2

In [9]:
%%time
collection_inverse_index = db.crawlerdb_INVERSE_INDEX
test1 = collection_inverse_index.find({"word": {"$in": tokenized_words}})
result_ids = set()
for i in test1:
    result_ids.update(i["documents"])

CPU times: user 3.3 ms, sys: 0 ns, total: 3.3 ms
Wall time: 19.5 ms


# in production file we use union atm

2

In [10]:
# %%time
# # combine result ids from inverse index with in a set via intersection
# #collection_inverse_index = db.crawlerdb_INVERSE_INDEX
# result_list_from_inverse_index = db.crawlerdb_INVERSE_INDEX.find({"word": {"$in": tokenized_words}}) ######################################################### tokenized und nicht stemmed
# try:
#     result_ids = set(result_list_from_inverse_index[0]["documents"])
#     for i in result_list_from_inverse_index:
#         result_ids = result_ids.intersection(i["documents"])
# except IndexError:
#     result_ids = set()

### holt sich die tf-idf werte zu den ids aus den inversen index (result_ids)

3

In [11]:
%%time
less_document_actually= matrix = vstack([result_dict_with_ids[i] for i in result_ids])

CPU times: user 14.4 ms, sys: 3.94 ms, total: 18.3 ms
Wall time: 17.5 ms


### Get from our TF-IDF - Vector the given important documents

### calculate tf-idf

4

In [12]:
%%time
cos_sim_results = sklearnPairwise.cosine_similarity(X=matrix,Y=tfidf_search_term).reshape(-1)

CPU times: user 4.19 ms, sys: 12 µs, total: 4.2 ms
Wall time: 3.09 ms


5

In [13]:
%%time
new_ids_with_scores = {}
i = 0
for a in result_ids:
    new_ids_with_scores[a] = cos_sim_results[i]
    i+=1

CPU times: user 399 µs, sys: 18 µs, total: 417 µs
Wall time: 422 µs


6

In [14]:
%%time
sorted_d = sorted(new_ids_with_scores.items(), key=operator.itemgetter(1), reverse=True)[0:20]
print(sorted_d)

[('5c30582988df4228e7867dd1', 0.49931165727219057), ('5c303e1c88df4228e7866bea', 0.3654545667638543), ('5c30128388df4228e7864deb', 0.3538282196534258), ('5c2ff4d088df4228e78639ee', 0.33912854975358947), ('5c303fd688df4228e7866d1c', 0.334818161485579), ('5c2fd41d88df4228e78623b6', 0.3293500593034371), ('5c3065b988df4228e78686e1', 0.31176762757845133), ('5c303e4688df4228e7866c08', 0.308486248326046), ('5c30207188df4228e786579e', 0.29276529806948226), ('5c30032d88df4228e7864393', 0.2917058867815179), ('5c2fc5b788df4228e7861a18', 0.28527732688503293), ('5c2fb3e888df422822371391', 0.2841450887408819), ('5c2ff32488df4228e78638d4', 0.2838054182836621), ('5c30585088df4228e7867de8', 0.2813745314582979), ('5c2fb40c88df422822372229', 0.2737912371506526), ('5c30462188df4228e7867177', 0.27212007492967977), ('5c303ee388df4228e7866c71', 0.2700960584205649), ('5c2fb3e888df4228223713e7', 0.26770139023395), ('5c3042bf88df4228e7866f1d', 0.26749673241313315), ('5c302fc988df4228e786622a', 0.264618216547761

7

In [15]:
%%time
ids = [ObjectId(sorted_d[i][0]) for i in range(len(sorted_d))]

CPU times: user 30 µs, sys: 1e+03 ns, total: 31 µs
Wall time: 33.6 µs


In [16]:
%%time
documents = collection.find({"_id" : {"$in" : ids}})
print(type(documents[0]))
documents[0]

<class 'dict'>
CPU times: user 1.83 ms, sys: 82 µs, total: 1.91 ms
Wall time: 2.1 ms


In [17]:
%%time
list_of_documents = list(documents)
for i in range(len(list_of_documents)):
    list_of_documents[i]["cossim"] = sorted_d[i][1]
list_of_documents

CPU times: user 796 µs, sys: 16 µs, total: 812 µs
Wall time: 1.25 ms
